In [ ]:
# Install rdflib-neo4j (do not forget to add the Unqiueness Constraint on the (r:Resource {r.uri}) property))
!pip install rdflib-neo4j

In [ ]:
from rdflib_neo4j import Neo4jStoreConfig, Neo4jStore, HANDLE_VOCAB_URI_STRATEGY
from rdflib import Namespace, Graph, URIRef, RDF, SKOS, Literal

In [ ]:
# Define your custom prefixes
prefixes = {
    'neo4ind': Namespace('http://neo4j.org/ind#'),
    'neo4voc': Namespace('http://neo4j.org/vocab/sw#'),
}

# Neo4j connection credentials
auth_data = {'uri': 'your_neo4j_uri',
                     'database': 'neo4j',
                     'user': "neo4j",
                     'pwd': 'password'}

# Define your Neo4jStoreConfig
config = Neo4jStoreConfig(auth_data=auth_data,
                          custom_prefixes=prefixes,
                          handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.IGNORE,
                          batching=False)

neo4j_store = Neo4jStore(config=config)
graph_store = Graph(store=neo4j_store)

# Import by reference, passing a url

In [ ]:
file_path = 'https://raw.githubusercontent.com/neo4j-labs/neosemantics/3.5/docs/rdf/nsmntx.ttl'
graph_store.parse(file_path,format="ttl")

# Writing individual triples using add

In [ ]:
aura = URIRef("http://neo4j.com/voc/tech#AuraDB")

graph_store.add((aura, RDF.type, SKOS.Concept))
graph_store.add((aura, SKOS.prefLabel, Literal("AuraDB")))
graph_store.add((aura, SKOS.broader, URIRef("http://neo4j.org/ind#neo4j355")))

# Import the RDF generated by a SPARQL query + Batching

In [ ]:
import requests
import urllib.parse

endpoint = "https://id.nlm.nih.gov/mesh/sparql"
sparql = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

CONSTRUCT { ?s ?p ?o }
FROM <http://id.nlm.nih.gov/mesh>
WHERE {
  {
    ?s ?p ?o
    filter(?s = mesh:D000086402 || ?o = mesh:D000086402)
   }
   union
  {
    mesh:D000086402 ?x ?s  .
    ?s ?p ?o .
    filter(?x != rdf:type && (isLiteral(?o) || ?p = rdf:type))
  }
   union
  {
    ?s ?x mesh:D000086402 .
    ?s ?p ?o .
    filter(isLiteral(?o|| ?p = rdf:type))
  }
}
"""

# Define your Neo4jStoreConfig
config = Neo4jStoreConfig(auth_data=auth_data,
                          custom_prefixes=prefixes,
                          handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.IGNORE,
                          batching=True)

neo4j_store = Neo4jStore(config=config)
graph_store = Graph(store=neo4j_store)

query_response = requests.get(endpoint, params = {"query": sparql , "format" : "TURTLE"})
graph_store.parse(data=query_response.text,format='ttl')
graph_store.close(commit_pending_transaction=True)